In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master("yarn").config('spark.executor.instances','100').appName("chiang_ct").getOrCreate()

In [8]:
%time df_cross= spark.read.csv("/user/spark/share/output/table1/cross_all",header=True)

CPU times: user 2.28 ms, sys: 447 µs, total: 2.73 ms
Wall time: 3.95 s


In [5]:
df_merge = spark.read.csv("/user/spark/share/output/merge_nc",header=True)

## fill na

In [10]:
df_cross.columns

['user_cate', 'pv', 'cart', 'fav', 'buy']

In [ ]:
from pyspark.sql.functions import isnull

In [9]:
for col in df_cross:
    print(df_cross.filter(isnull(col)).count(),"----->",col)

0 -----> Column<b'user_cate'>
727585 -----> Column<b'pv'>
48981068 -----> Column<b'cart'>
52764629 -----> Column<b'fav'>
51300144 -----> Column<b'buy'>


In [11]:
df_cross = df_cross.fillna("0")

In [12]:
for col in df_cross:
    print(df_cross.filter(isnull(col)).count(),"----->",col)

0 -----> Column<b'user_cate'>
0 -----> Column<b'pv'>
0 -----> Column<b'cart'>
0 -----> Column<b'fav'>
0 -----> Column<b'buy'>


In [20]:
df_cross.count()

57259092

## join cross with merge

In [19]:
df_cross.columns

['user_cate_c', 'pv', 'cart', 'fav', 'buy']

In [18]:
df_cross = df_cross.selectExpr('user_cate as user_cate_c', 'pv', 'cart', 'fav', 'buy')

In [21]:
table1 = df_merge.join(df_cross,(df_merge.user_cate == df_cross.user_cate_c) ,'left_outer')

In [22]:
table1.count()

26557961

In [23]:
for col in table1:
    print(table1.filter(isnull(col)).count(),"----->",col)

0 -----> Column<b'user'>
0 -----> Column<b'time_stamp'>
0 -----> Column<b'adgroup_id'>
0 -----> Column<b'pid'>
0 -----> Column<b'clk'>
0 -----> Column<b'cate_id'>
0 -----> Column<b'campaign_id'>
0 -----> Column<b'customer'>
0 -----> Column<b'brand'>
0 -----> Column<b'price'>
1528526 -----> Column<b'cms_segid'>
1528526 -----> Column<b'cms_group_id'>
1528526 -----> Column<b'final_gender_code'>
1528526 -----> Column<b'age_level'>
14552798 -----> Column<b'pvalue_level'>
1528526 -----> Column<b'shopping_level'>
1528526 -----> Column<b'occupation'>
8224829 -----> Column<b'new_user_class_level'>
0 -----> Column<b'user_cate'>
1141355 -----> Column<b'user_cate_c'>
1141355 -----> Column<b'pv'>
1141355 -----> Column<b'cart'>
1141355 -----> Column<b'fav'>
1141355 -----> Column<b'buy'>


In [25]:
table1.columns

['user',
 'time_stamp',
 'adgroup_id',
 'pid',
 'clk',
 'cate_id',
 'campaign_id',
 'customer',
 'brand',
 'price',
 'cms_segid',
 'cms_group_id',
 'final_gender_code',
 'age_level',
 'pvalue_level',
 'shopping_level',
 'occupation',
 'new_user_class_level',
 'user_cate',
 'user_cate_c',
 'pv',
 'cart',
 'fav',
 'buy']

In [26]:
table1.show()

+-------+----------+----------+-----------+---+-------+-----------+--------+------+------+---------+------------+-----------------+---------+------------+--------------+----------+--------------------+-------------+-------------+----+----+----+----+
|   user|time_stamp|adgroup_id|        pid|clk|cate_id|campaign_id|customer| brand| price|cms_segid|cms_group_id|final_gender_code|age_level|pvalue_level|shopping_level|occupation|new_user_class_level|    user_cate|  user_cate_c|  pv|cart| fav| buy|
+-------+----------+----------+-----------+---+-------+-----------+--------+------+------+---------+------------+-----------------+---------+------------+--------------+----------+--------------------+-------------+-------------+----+----+----+----+
|1000080|1494500915|    282550|430539_1007|  1|  10965|     240785|  130530|306014| 130.0|        0|           3|                2|        3|           1|             3|         0|                null|1000080_10965|1000080_10965|  10|   0|   0|   0|


In [28]:
cross = ['pv', 'cart', 'fav', 'buy']
table1_f = table1
for col in cross:
    table1_f = table1_f.fillna({col:0})

In [29]:
for col in table1_f:
    print(table1_f.filter(isnull(col)).count(),"----->",col)

0 -----> Column<b'user'>
0 -----> Column<b'time_stamp'>
0 -----> Column<b'adgroup_id'>
0 -----> Column<b'pid'>
0 -----> Column<b'clk'>
0 -----> Column<b'cate_id'>
0 -----> Column<b'campaign_id'>
0 -----> Column<b'customer'>
0 -----> Column<b'brand'>
0 -----> Column<b'price'>
1528526 -----> Column<b'cms_segid'>
1528526 -----> Column<b'cms_group_id'>
1528526 -----> Column<b'final_gender_code'>
1528526 -----> Column<b'age_level'>
14552798 -----> Column<b'pvalue_level'>
1528526 -----> Column<b'shopping_level'>
1528526 -----> Column<b'occupation'>
8224829 -----> Column<b'new_user_class_level'>
0 -----> Column<b'user_cate'>
1141355 -----> Column<b'user_cate_c'>
0 -----> Column<b'pv'>
0 -----> Column<b'cart'>
0 -----> Column<b'fav'>
0 -----> Column<b'buy'>


In [30]:
table1_f = table1_f.fillna('999999999')

In [47]:
table1_f.filter(table1_f.brand=='NULL').count()  ##just checking

8291694

In [33]:
table1_f = table1_f.drop('user_cate_c')

In [44]:
table1 = table1.drop('user_cate_c')

In [35]:
table1_f.columns

['user',
 'time_stamp',
 'adgroup_id',
 'pid',
 'clk',
 'cate_id',
 'campaign_id',
 'customer',
 'brand',
 'price',
 'cms_segid',
 'cms_group_id',
 'final_gender_code',
 'age_level',
 'pvalue_level',
 'shopping_level',
 'occupation',
 'new_user_class_level',
 'user_cate',
 'pv',
 'cart',
 'fav',
 'buy']

In [36]:
for col in table1_f:
    print(table1_f.filter(isnull(col)).count(),"----->",col)

0 -----> Column<b'user'>
0 -----> Column<b'time_stamp'>
0 -----> Column<b'adgroup_id'>
0 -----> Column<b'pid'>
0 -----> Column<b'clk'>
0 -----> Column<b'cate_id'>
0 -----> Column<b'campaign_id'>
0 -----> Column<b'customer'>
0 -----> Column<b'brand'>
0 -----> Column<b'price'>
0 -----> Column<b'cms_segid'>
0 -----> Column<b'cms_group_id'>
0 -----> Column<b'final_gender_code'>
0 -----> Column<b'age_level'>
0 -----> Column<b'pvalue_level'>
0 -----> Column<b'shopping_level'>
0 -----> Column<b'occupation'>
0 -----> Column<b'new_user_class_level'>
0 -----> Column<b'user_cate'>
0 -----> Column<b'pv'>
0 -----> Column<b'cart'>
0 -----> Column<b'fav'>
0 -----> Column<b'buy'>


In [45]:
%time table1.write.csv("hdfs:///user/spark/share/output/table1/table1",header="true")

CPU times: user 8.51 ms, sys: 7.5 ms, total: 16 ms
Wall time: 56.4 s


In [37]:
%time table1_f.write.csv("hdfs:///user/spark/share/output/table1/table1_f",header="true")

CPU times: user 10.2 ms, sys: 0 ns, total: 10.2 ms
Wall time: 1min 4s
